In [ ]:
#Import required modules

import torch
import torchvision
import os

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/Python_Colab/AR_hand_recognition")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Define custom dataloaders

class HandDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = './'+self.img_labels.iloc[idx, 0].replace('\\','/')
        image = Image.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset    = HandDataset('./data/trainDataSmall.csv', transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True)

testset    = HandDataset('./data/testDataSmall.csv', transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=True)

In [ ]:
#Define the neural network architecture

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 4) 
        self.pool = nn.MaxPool2d(2) 
        self.conv2 = nn.Conv2d(6, 16, 4) 
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 48)
        self.fc4 = nn.Linear(48, 16)
        self.fc5 = nn.Linear(16, 1)

    def forward(self, x):
        #CNN:

        #x (3 colour channels, 25x25 image) (excuding batch dimension)        => x.shape = 03x25x25 
        x = self.conv1(x) #6 feature maps (output channels), 4x4 kernal       => x.shape = 06x22x22
        x = F.relu(x) #returns input tensor with all negative values set to 0 => x.shape = 06x22x22
        x = self.pool(x) #2x2 kernal returning max value each step:           => x.shape = 06x11x11
        x = self.conv2(x) #16 output channels, 4x4 kernal                     => x.shape = 16x08x08
        x = F.relu(x) #                                                       => x.shape = 16x08x08
        x = self.pool(x) #                                                    => x.shape = 16x04x04
        x = torch.flatten(x, 1) #flatten all dimensions except batch          => x.shape = 256

        #FFNN
        x = F.relu(self.fc1(x)) #256 input nodes to 120 hidden nodes          => x.shape = 120
        x = F.relu(self.fc2(x)) #120 hidden nodes to 84 hidden nodes          => x.shape = 84
        x = F.relu(self.fc3(x)) #84 hidden nodes to 48 hidden nodes           => x.shape = 84
        x = F.relu(self.fc4(x)) #48 hidden nodes to 16 hidden nodes           => x.shape = 84
        x = torch.sigmoid(self.fc5(x))        #16 hidden nodes to 1 ouput node (overkill?)  => x.shape = 1
        return x


net = nn.DataParallel(Net().cuda())

In [ ]:
#Optional, load a pre-trained model

PATH = './' + input('Name of network? ') + '.pth'

net = Net()
net.load_state_dict(torch.load(PATH))
net = nn.DataParallel(net.cuda())

print('Loaded Network')

Name of network? handNetAlt
Loaded Network


In [ ]:
def LoadNetTXT(model, path):
	data_dict = {}
	fin = open(path, 'r')
	i = 0
	odd = 1
	prev_key = None
	while True:
		s = fin.readline().strip()
		if not s: break
		elif odd: prev_key = s
		else:
			val = eval(s)
			if type(val) != type([]): data_dict[prev_key] = torch.FloatTensor([eval(s)])[0]
			else:                     data_dict[prev_key] = torch.FloatTensor(eval(s))
			i += 1
		odd = (odd + 1) % 2

	# Replace existing values with loaded
	own_state = model.state_dict()
	for k, v in data_dict.items():
		if k in own_state:
			try: own_state[k].copy_(v)
			except: sys.exit(0)


PATH = './' + input('Name of network? ') + '.txt'

net = Net()
LoadNetTXT(net, PATH)
net = nn.DataParallel(net.cuda())

print('Loaded Network')

Name of network? handNet
Loaded Network


In [ ]:
#Compare the network's predictions to the ground truth

def imshow(img):
    img = (img + 1) / 2     # unnormalize the funky way
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def TestNetwork(testloader, net, just_one=True):

    dataiter = testloader
    if just_one: dataiter = [iter(testloader)]

    fails = 0
    total = 0

    for batch in dataiter:

      if just_one: images, labels = batch.next()
      else:        images, labels = batch

      if just_one:
        while 1 not in [int(labels[j]) for j in range(4)]:
            batch = iter(batch)
            images, labels = batch.next()

      outputs = net(images)

      if just_one:
        imshow(torchvision.utils.make_grid(images))
        print('Ground Truth: ', ' '.join('%5s' % int(labels[j]) for j in range(len(labels))))
        print('Predicted:    ', ' '.join('%5s' % float(outputs[j]) for j in range(len(outputs))))
      
      fails += sum([abs(int(labels[j]) - round(float(outputs[j]))) for j in range(len(outputs))])
      total += 4
    
    print(str(total - fails) + '/' + str(total), '-', str(round( (total - fails) / total * 100, 2)) + '% accurate')


TestNetwork(testloader, net, just_one=False)

115/124 - 92.74% accurate


In [ ]:
#Train the network

criterion = nn.DataParallel(nn.BCELoss().cuda())
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.float().cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    if epoch % 10 == 0:
        print('loss:', running_loss / i)
        TestNetwork(testloader, net, just_one=False)
        running_loss = 0.0

print('Finished Training Network')

In [ ]:
#Save model

PATH = './' + input('Name for network? ') + '.pth'

net.cpu()
torch.save(net.module.state_dict(), PATH)
net.cuda()

print('Saved')

In [ ]:
#Save the network in TXT format so older pytorch versions can load it (ie torch-1.0.0, which I am using on my rPI4)

def SaveNetTXT(model, path):
    fout = open(path, 'w')
    for k, v in model.module.state_dict().items():
        fout.write(str(k) + '\n')
        fout.write(str(v.tolist()) + '\n')
    fout.close()

PATH = './' + input('Name for network? ') + '.txt'

net.cpu()
SaveNetTXT(net, PATH)
net.cuda()

print('Saved')

Name for network? handNetAlt
Saved
